In [1]:
! pip install numpy pandas tensorflow

  Using cached numpy-2.3.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached pandas-2.3.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
  Using cached tensorflow-2.19.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached absl_py-2.3.0-py3-none-any.whl.metadata (2.4 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached flatbuffers-25.2.10-py2.py3-none-any.whl.metadata (875 bytes)
  Using cached gast-0.6.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-manylinux2010_x86_64.whl.metadata (5.2 kB)
  Using cached opt_einsum-3.4.0-py3-none-any.whl.metadata (6.3 kB)
  Using cached protobuf-5.29.5-cp38-abi3-manylinux2014_x86_64.whl.metada

In [217]:
! pip install -U scikit-learn

  Using cached scikit_learn-1.7.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (17 kB)
  Using cached scipy-1.15.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.7.0-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.5 MB)
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
Using cached scipy-1.15.3-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (37.3 MB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-learn] [scikit-learn]


In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM, Embedding, Bidirectional, TextVectorization, Input, TimeDistributed
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import ast
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
dataset = pd.read_csv('data/ner.csv')

In [4]:
max_len = 100

In [5]:
X = dataset['Sentence'].values
X

array(['Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .',
       'Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as " Bush Number One Terrorist " and " Stop the Bombings . "',
       'They marched from the Houses of Parliament to a rally in Hyde Park .',
       ..., 'Two more landed in fields belonging to a nearby village .',
       'They say not all of the rockets exploded upon impact .',
       'Indian forces said they responded to the attack'], dtype=object)

In [6]:

Y = dataset['Tag'].apply(ast.literal_eval).tolist()

In [7]:
Y

[['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-gpe',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-per',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'I-geo',
  'O'],
 ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'B-org',
  'I-org',
  'O',
  'O',
  'O',
  'B-gpe',
  'O',
  'O',
  'O',
  'B-geo',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'B-gpe',
  'O',
  'O',
  'O',
  'O',
  'B-geo',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-gpe',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'B-geo',
  'O',
  'O',
  'O',
  '

<h2>Creating index to tag and tag to index dictionary</h2>
<p>The below code first takes all the unique tags from the label and then creates 2 dictionaries (tag2idx and idx2tag)</p>
<p>The dictionary tag2idx matches tags with its respective index.</p>
<p>The dictionary idx2tag matches index with its respective tag.</p>

In [8]:
unique_tags = list(set(tag for seq in Y for tag in seq))
tag2idx = {tag: i + 1 for i, tag in enumerate(unique_tags)}
tag2idx['PAD'] = 0
idx2tag = {i: tag for tag, i in tag2idx.items()}

In [9]:
tag2idx

{'B-geo': 1,
 'B-eve': 2,
 'B-nat': 3,
 'I-org': 4,
 'I-tim': 5,
 'I-gpe': 6,
 'B-tim': 7,
 'B-org': 8,
 'I-eve': 9,
 'B-art': 10,
 'I-nat': 11,
 'I-geo': 12,
 'B-gpe': 13,
 'O': 14,
 'B-per': 15,
 'I-per': 16,
 'I-art': 17,
 'PAD': 0}

<h2>Padding and one hot encoding</h2>
<p>The below code initially converts all the tags to its index list.</p>
<p>Then all the list is normalized to same length by adding 'PAD' at end </p>
<p>Then One hot encoding is done</p>

In [10]:
label_seqs = [[tag2idx[tag] for tag in seq] for seq in Y]
label_seqs = pad_sequences(label_seqs, maxlen=max_len, padding='post')
label_seqs = to_categorical(label_seqs, num_classes=len(tag2idx))

In [11]:
vectorizer = TextVectorization(output_mode='int', output_sequence_length=max_len)
vectorizer.adapt(X)

# Convert raw sentences to integer sequences
X_seq = vectorizer(tf.constant(X))
vocab_size = len(vectorizer.get_vocabulary())

E0000 00:00:1750341989.339278     829 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1750341989.342915     829 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [12]:
model = Sequential()
model.add(Input(shape=(max_len,), dtype=tf.int32))
model.add(Embedding(input_dim=len(vectorizer.get_vocabulary()), output_dim=128, input_length=100))
model.add(Bidirectional(LSTM(64, return_sequences=True)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(tag2idx), activation='softmax'))

/run/media/aadimprajapati/New Volume/fnclick/python/nlp/cbr-bilstm-ner/env/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [13]:
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 100, 128)       │     4,007,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 100, 128)       │        98,816 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100, 64)        │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100, 64)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 100, 18)        │         1,170 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,115,794 (15.70 MB)

 Trainable params: 4,115,794 (15.70 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model.fit(X_seq, label_seqs, epochs=10, batch_size=32, validation_split=0.2, callbacks=[stopping])

Epoch 1/10
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 51s 41ms/step - accuracy: 0.8667 - loss: 0.6828 - val_accuracy: 0.9593 - val_loss: 0.1758
Epoch 2/10
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 47s 39ms/step - accuracy: 0.9588 - loss: 0.1920 - val_accuracy: 0.9608 - val_loss: 0.1480
Epoch 3/10
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 45s 37ms/step - accuracy: 0.9601 - loss: 0.1592 - val_accuracy: 0.9609 - val_loss: 0.1382
Epoch 4/10
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 46s 38ms/step - accuracy: 0.9613 - loss: 0.1445 - val_accuracy: 0.9614 - val_loss: 0.1341
Epoch 5/10
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 45s 38ms/step - accuracy: 0.9621 - loss: 0.1360 - val_accuracy: 0.9625 - val_loss: 0.1282
Epoch 6/10
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 46s 38ms/step - accuracy: 0.9632 - loss: 0.1291 - val_accuracy: 0.9638 - val_loss: 0.1230
Epoch 7/10
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 49s 40ms/step - accuracy: 0.9654 - loss: 0.1192 - val_accuracy: 0.9661 - val_loss: 0.1170
Epoch 8/10
1199/1199 ━━━━━━━━━━━━━━━━━━━━ 48s 40ms/step - accuracy: 0.9668 -

In [16]:
sentence = "Barack Obama was born in Hawaii."
sentence_seq = vectorizer(tf.constant([sentence]))
sentence_seq = pad_sequences(sentence_seq.numpy(), maxlen=max_len, padding='post')
predictions = model.predict(sentence_seq)
predicted_tags = np.argmax(predictions, axis=-1)[0]
predicted_tags = [idx2tag[i] for i in predicted_tags if i != 0]
for word, tag in zip(sentence.split(), predicted_tags):
    print(f"{word}: {tag}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step
Barack: B-per
Obama: I-per
was: O
born: O
in: O
Hawaii.: O
